## 임포트

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.by import By

import re
import time

In [2]:
# 엑셀에 여러 시트로 저장하기 위한 모듈
!pip install Xlsxwriter

## 데이터수집

In [6]:
driver = webdriver.Chrome("C:/Users/wbs04/workspace/chromedriver_win32/chromedriver.exe")

C:\Users\wbs04\AppData\Local\Temp\ipykernel_8072\3636124997.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:/Users/wbs04/workspace/chromedriver_win32/chromedriver.exe")


### 해당 url(기록실)로 이동

In [22]:
# 해당 url로 이동
url = "https://www.koreabaseball.com"
driver.get(url)

In [23]:
go_to_page('#lnb > li:nth-child(3) > a')

In [24]:
go_to_page('#lnb02')

### 함수정의

In [25]:
def go_to_page(selector):
    global soup
    click_on = driver.find_element(By.CSS_SELECTOR, selector)
    click_on.click()
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

In [26]:
def make_dataframe():
    col_names = []
    record_col = soup.select("#cphContents_cphContents_cphContents_udpContent > div.record_result > table > thead > tr > th")
    for item in record_col:
        col_names.append(item.text)
        
    return pd.DataFrame(columns=col_names)

In [27]:
def team_record_dict():
    # 전체 팀에 대해 자료 가져오기(년도별)
    team_dict = {}
    teams = soup.select('#cphContents_cphContents_cphContents_ddlTeam_ddlTeam > option')
    for idx, team in enumerate(teams[1:]):
        df_record = make_dataframe()

        team_name = team.text
        print(idx+1, team_name)  # 스크래핑 완료 확인용
        go_to_page(f'#cphContents_cphContents_cphContents_ddlTeam_ddlTeam > option:nth-child({idx+2})')

        # 페이지 길이
        # 페이지 번호 길이
        page_btn = soup.select("div.paging > a")     
        page_range = 2 if len(page_btn) == 1 else len(page_btn)-1

        for j in range(1, page_range):
            # 페이지 이동
            go_to_page(f'#cphContents_cphContents_cphContents_ucPager_btnNo{j}')

            # 데이터프레임에 선수 정보 한 행씩 추가
            players_list = soup.select('#cphContents_cphContents_cphContents_udpContent > div.record_result > table > tbody > tr')
            for player in players_list:
                player_record = player.text.strip().split('\n')
                df_record.loc[len(df_record)] = player_record

        team_dict[team_name] = df_record
    return team_dict

In [28]:
def save_to_excel(dictionary, foler_name, *details):
    for k, v in dictionary.items():
        if len(details):
            writer = pd.ExcelWriter(f'{foler_name}/{k}_details.xlsx', engine='xlsxwriter')
        else:
            writer = pd.ExcelWriter(f'{foler_name}/{k}.xlsx', engine='xlsxwriter')

        for item in v.keys():
            # 시트에 팀별로 저장
            dictionary[k][item].to_excel(writer, sheet_name=item, index=False)
        # 파일 저장
        writer.save()

### 타자 기록

#### 기본기록

In [29]:
df_record = make_dataframe()

years = soup.select('#cphContents_cphContents_cphContents_ddlSeason_ddlSeason > option')
hitter_dict = {}
for idx, year in enumerate(years[:-1]):
    season = year.text
    print(f"{season}시즌")  # 스크래핑 완료 확인용
    go_to_page(f'#cphContents_cphContents_cphContents_ddlSeason_ddlSeason > option:nth-child({idx+1})')
    
    hitter_dict[season] = team_record_dict()

1982시즌
1 롯데  
2 삼미  
3 삼성  
4 해태  
5 MBC   
6 OB    
1983시즌
1 롯데  
2 삼미  
3 삼성  
4 해태  
5 MBC   
6 OB    
1984시즌
1 롯데  
2 삼미  
3 삼성  
4 해태  
5 MBC   
6 OB    
1985시즌
1 롯데
2 삼성  
3 청보
4 해태
5 MBC
6 OB
1986시즌
1 롯데
2 빙그레
3 삼성
4 청보
5 해태
6 MBC
7 OB
1987시즌
1 롯데
2 빙그레
3 삼성
4 청보
5 해태
6 MBC
7 OB
1988시즌
1 롯데
2 빙그레
3 삼성
4 태평양
5 해태
6 MBC
7 OB
1989시즌
1 롯데
2 빙그레
3 삼성
4 태평양
5 해태
6 MBC
7 OB
1990시즌
1 롯데
2 빙그레
3 삼성
4 태평양
5 해태
6 LG
7 OB
1991시즌
1 롯데
2 빙그레
3 삼성
4 쌍방울
5 태평양
6 해태
7 LG
8 OB
1992시즌
1 롯데
2 빙그레
3 삼성
4 쌍방울
5 태평양
6 해태
7 LG
8 OB
1993시즌
1 롯데
2 빙그레
3 삼성
4 쌍방울
5 태평양
6 해태
7 LG
8 OB
1994시즌
1 롯데
2 삼성
3 쌍방울
4 태평양
5 한화
6 해태
7 LG
8 OB
1995시즌
1 롯데
2 삼성
3 쌍방울
4 태평양
5 한화
6 해태
7 LG
8 OB
1996시즌
1 롯데
2 삼성
3 쌍방울
4 한화
5 해태
6 현대
7 LG
8 OB
1997시즌
1 롯데
2 삼성
3 쌍방울
4 한화
5 해태
6 현대
7 LG
8 OB
1998시즌
1 롯데
2 삼성
3 쌍방울
4 한화
5 해태
6 현대
7 LG
8 OB
1999시즌
1 두산
2 롯데
3 삼성
4 쌍방울
5 한화
6 해태
7 현대
8 LG
2000시즌
1 두산
2 롯데
3 삼성
4 한화
5 해태
6 현대
7 LG
8 SK
2001시즌
1 두산
2 롯데
3 삼성
4 한화
5 현대
6 KIA
7 LG
8 SK
2002시즌
1 두산
2 롯데
3 삼성
4 한화
5 현대
6 KIA
7 LG
8

In [30]:
save_to_excel(hitter_dict, 'hitter_record2')

#### 세부기록

In [31]:
go_to_page('#cphContents_cphContents_cphContents_udpContent > div.row > div.tab-depth3 > ul > li:nth-child(2) > a')

In [32]:
df_record = make_dataframe()

years = soup.select('#cphContents_cphContents_cphContents_ddlSeason_ddlSeason > option')
hitter_dict = {}
for idx, year in enumerate(years[-21:]):
    season = year.text
    print(f"{season}시즌")  # 스크래핑 완료 확인용
    go_to_page(f'#cphContents_cphContents_cphContents_ddlSeason_ddlSeason > option:nth-child({idx+21})')
    
    hitter_dict[season] = team_record_dict()

2002시즌
1 두산
2 롯데
3 삼성
4 키움
5 한화
6 KIA
7 KT
8 LG
9 NC
10 SSG
2003시즌
1 두산
2 롯데
3 삼성
4 키움
5 한화
6 KIA
7 KT
8 LG
9 NC
10 SSG
2004시즌
1 두산
2 롯데
3 삼성
4 키움
5 한화
6 KIA
7 KT
8 LG
9 NC
10 SSG
2005시즌
1 두산
2 롯데
3 삼성
4 키움
5 한화
6 KIA
7 KT
8 LG
9 NC
10 SSG
2006시즌
1 두산
2 롯데
3 삼성
4 키움
5 한화
6 KIA
7 KT
8 LG
9 NC
10 SSG
2007시즌
1 두산
2 롯데
3 삼성
4 키움
5 한화
6 KIA
7 KT
8 LG
9 NC
10 SSG
2008시즌
1 두산
2 롯데
3 삼성
4 키움
5 한화
6 KIA
7 KT
8 LG
9 NC
10 SSG
2009시즌
1 두산
2 롯데
3 삼성
4 키움
5 한화
6 KIA
7 KT
8 LG
9 NC
10 SSG
2010시즌
1 두산
2 롯데
3 삼성
4 키움
5 한화
6 KIA
7 KT
8 LG
9 NC
10 SSG
2011시즌
1 두산
2 롯데
3 삼성
4 키움
5 한화
6 KIA
7 KT
8 LG
9 NC
10 SSG
2012시즌
1 두산
2 롯데
3 삼성
4 키움
5 한화
6 KIA
7 KT
8 LG
9 NC
10 SSG
2013시즌
1 두산
2 롯데
3 삼성
4 키움
5 한화
6 KIA
7 KT
8 LG
9 NC
10 SSG
2014시즌
1 두산
2 롯데
3 삼성
4 키움
5 한화
6 KIA
7 KT
8 LG
9 NC
10 SSG
2015시즌
1 두산
2 롯데
3 삼성
4 키움
5 한화
6 KIA
7 KT
8 LG
9 NC
10 SSG
2016시즌
1 두산
2 롯데
3 삼성
4 키움
5 한화
6 KIA
7 KT
8 LG
9 NC
10 SSG
2017시즌
1 두산
2 롯데
3 삼성
4 키움
5 한화
6 KIA
7 KT
8 LG
9 NC
10 SSG
2018시즌
1 두산
2 롯데
3 삼성
4 키움
5 한화
6 KIA
7 

In [33]:
save_to_excel(hitter_dict, 'hitter_record2', 'details')

### 투수 기록

In [34]:
# 투수로 이동
go_to_page('#contents > div.sub-content > div.tab-depth2 > ul > li:nth-child(2)')

#### 기본기록

In [35]:
df_record = make_dataframe()

years = soup.select('#cphContents_cphContents_cphContents_ddlSeason_ddlSeason > option')
pitcher_dict = {}
for idx, year in enumerate(years[:-1]):
    season = year.text
    print(f"{season}시즌")  # 스크래핑 완료 확인용
    go_to_page(f'#cphContents_cphContents_cphContents_ddlSeason_ddlSeason > option:nth-child({idx+1})')
    
    pitcher_dict[season] = team_record_dict()

1982시즌
1 롯데  
2 삼미  
3 삼성  
4 해태  
5 MBC   
6 OB    
1983시즌
1 롯데  
2 삼미  
3 삼성  
4 해태  
5 MBC   
6 OB    
1984시즌
1 롯데  
2 삼미  
3 삼성  
4 해태  
5 MBC   
6 OB    
1985시즌
1 롯데
2 삼성  
3 청보
4 해태
5 MBC
6 OB
1986시즌
1 롯데
2 빙그레
3 삼성
4 청보
5 해태
6 MBC
7 OB
1987시즌
1 롯데
2 빙그레
3 삼성
4 청보
5 해태
6 MBC
7 OB
1988시즌
1 롯데
2 빙그레
3 삼성
4 태평양
5 해태
6 MBC
7 OB
1989시즌
1 롯데
2 빙그레
3 삼성
4 태평양
5 해태
6 MBC
7 OB
1990시즌
1 롯데
2 빙그레
3 삼성
4 태평양
5 해태
6 LG
7 OB
1991시즌
1 롯데
2 빙그레
3 삼성
4 쌍방울
5 태평양
6 해태
7 LG
8 OB
1992시즌
1 롯데
2 빙그레
3 삼성
4 쌍방울
5 태평양
6 해태
7 LG
8 OB
1993시즌
1 롯데
2 빙그레
3 삼성
4 쌍방울
5 태평양
6 해태
7 LG
8 OB
1994시즌
1 롯데
2 삼성
3 쌍방울
4 태평양
5 한화
6 해태
7 LG
8 OB
1995시즌
1 롯데
2 삼성
3 쌍방울
4 태평양
5 한화
6 해태
7 LG
8 OB
1996시즌
1 롯데
2 삼성
3 쌍방울
4 한화
5 해태
6 현대
7 LG
8 OB
1997시즌
1 롯데
2 삼성
3 쌍방울
4 한화
5 해태
6 현대
7 LG
8 OB
1998시즌
1 롯데
2 삼성
3 쌍방울
4 한화
5 해태
6 현대
7 LG
8 OB
1999시즌
1 두산
2 롯데
3 삼성
4 쌍방울
5 한화
6 해태
7 현대
8 LG
2000시즌
1 두산
2 롯데
3 삼성
4 한화
5 해태
6 현대
7 LG
8 SK
2001시즌
1 두산
2 롯데
3 삼성
4 한화
5 현대
6 KIA
7 LG
8 SK
2002시즌
1 두산
2 롯데
3 삼성
4 한화
5 현대
6 KIA
7 LG
8

In [36]:
save_to_excel(pitcher_dict, 'pitcher_record2')

#### 세부기록

In [37]:
go_to_page('#cphContents_cphContents_cphContents_udpContent > div.row > div.tab-depth3 > ul > li:nth-child(2) > a')

In [38]:
df_record = make_dataframe()

years = soup.select('#cphContents_cphContents_cphContents_ddlSeason_ddlSeason > option')
pitcher_dict = {}
for idx, year in enumerate(years[-21:]):
    season = year.text
    print(f"{season}시즌")  # 스크래핑 완료 확인용
    go_to_page(f'#cphContents_cphContents_cphContents_ddlSeason_ddlSeason > option:nth-child({idx+21})')
    
    pitcher_dict[season] = team_record_dict()

2002시즌
1 두산
2 롯데
3 삼성
4 한화
5 현대
6 KIA
7 LG
8 SK
2003시즌
1 두산
2 롯데
3 삼성
4 한화
5 현대
6 KIA
7 LG
8 SK
2004시즌
1 두산
2 롯데
3 삼성
4 한화
5 현대
6 KIA
7 LG
8 SK
2005시즌
1 두산
2 롯데
3 삼성
4 한화
5 현대
6 KIA
7 LG
8 SK
2006시즌
1 두산
2 롯데
3 삼성
4 한화
5 현대
6 KIA
7 LG
8 SK
2007시즌
1 두산
2 롯데
3 삼성
4 한화
5 현대
6 KIA
7 LG
8 SK
2008시즌
1 두산
2 롯데
3 삼성
4 우리
5 한화
6 KIA
7 LG
8 SK
2009시즌
1 히어로즈
2 두산
3 롯데
4 삼성
5 한화
6 KIA
7 LG
8 SK
2010시즌
1 넥센
2 두산
3 롯데
4 삼성
5 한화
6 KIA
7 LG
8 SK
2011시즌
1 넥센
2 두산
3 롯데
4 삼성
5 한화
6 KIA
7 LG
8 SK
2012시즌
1 넥센
2 두산
3 롯데
4 삼성
5 한화
6 KIA
7 LG
8 SK
2013시즌
1 넥센
2 두산
3 롯데
4 삼성
5 한화
6 KIA
7 LG
8 NC
9 SK
2014시즌
1 넥센
2 두산
3 롯데
4 삼성
5 한화
6 KIA
7 LG
8 NC
9 SK
2015시즌
1 넥센
2 두산
3 롯데
4 삼성
5 한화
6 KIA
7 KT
8 LG
9 NC
10 SK
2016시즌
1 넥센
2 두산
3 롯데
4 삼성
5 한화
6 KIA
7 KT
8 LG
9 NC
10 SK
2017시즌
1 넥센
2 두산
3 롯데
4 삼성
5 한화
6 KIA
7 KT
8 LG
9 NC
10 SK
2018시즌
1 넥센
2 두산
3 롯데
4 삼성
5 한화
6 KIA
7 KT
8 LG
9 NC
10 SK
2019시즌
1 두산
2 롯데
3 삼성
4 키움
5 한화
6 KIA
7 KT
8 LG
9 NC
10 SK
2020시즌
1 두산
2 롯데
3 삼성
4 키움
5 한화
6 KIA
7 KT
8 LG
9 NC
10 SK
2021시즌
1 두

In [39]:
save_to_excel(pitcher_dict, 'pitcher_record2', 'details')

### 수비 기록

In [40]:
# 수비로 이동
go_to_page('#contents > div.sub-content > div.tab-depth2 > ul > li:nth-child(3)')

In [41]:
df_record = make_dataframe()

years = soup.select('#cphContents_cphContents_cphContents_ddlSeason_ddlSeason > option')
defence_dict = {}
for idx, year in enumerate(years):
    season = year.text
    print(f"{season}시즌")  # 스크래핑 완료 확인용
    go_to_page(f'#cphContents_cphContents_cphContents_ddlSeason_ddlSeason > option:nth-child({idx+1})')
    
    defence_dict[season] = team_record_dict()

2001시즌
1 두산
2 롯데
3 삼성
4 한화
5 현대
6 KIA
7 LG
8 SK
2002시즌
1 두산
2 롯데
3 삼성
4 한화
5 현대
6 KIA
7 LG
8 SK
2003시즌
1 두산
2 롯데
3 삼성
4 한화
5 현대
6 KIA
7 LG
8 SK
2004시즌
1 두산
2 롯데
3 삼성
4 한화
5 현대
6 KIA
7 LG
8 SK
2005시즌
1 두산
2 롯데
3 삼성
4 한화
5 현대
6 KIA
7 LG
8 SK
2006시즌
1 두산
2 롯데
3 삼성
4 한화
5 현대
6 KIA
7 LG
8 SK
2007시즌
1 두산
2 롯데
3 삼성
4 한화
5 현대
6 KIA
7 LG
8 SK
2008시즌
1 두산
2 롯데
3 삼성
4 우리
5 한화
6 KIA
7 LG
8 SK
2009시즌
1 히어로즈
2 두산
3 롯데
4 삼성
5 한화
6 KIA
7 LG
8 SK
2010시즌
1 넥센
2 두산
3 롯데
4 삼성
5 한화
6 KIA
7 LG
8 SK
2011시즌
1 넥센
2 두산
3 롯데
4 삼성
5 한화
6 KIA
7 LG
8 SK
2012시즌
1 넥센
2 두산
3 롯데
4 삼성
5 한화
6 KIA
7 LG
8 SK
2013시즌
1 넥센
2 두산
3 롯데
4 삼성
5 한화
6 KIA
7 LG
8 NC
9 SK
2014시즌
1 넥센
2 두산
3 롯데
4 삼성
5 한화
6 KIA
7 LG
8 NC
9 SK
2015시즌
1 넥센
2 두산
3 롯데
4 삼성
5 한화
6 KIA
7 KT
8 LG
9 NC
10 SK
2016시즌
1 넥센
2 두산
3 롯데
4 삼성
5 한화
6 KIA
7 KT
8 LG
9 NC
10 SK
2017시즌
1 넥센
2 두산
3 롯데
4 삼성
5 한화
6 KIA
7 KT
8 LG
9 NC
10 SK
2018시즌
1 넥센
2 두산
3 롯데
4 삼성
5 한화
6 KIA
7 KT
8 LG
9 NC
10 SK
2019시즌
1 두산
2 롯데
3 삼성
4 키움
5 한화
6 KIA
7 KT
8 LG
9 NC
10 SK
2020시즌
1 두산
2 롯데
3 삼성

In [42]:
save_to_excel(defence_dict, 'defence_record2')

### 도루 기록

In [44]:
# 도루로 이동
go_to_page('#contents > div.sub-content > div.tab-depth2 > ul > li:nth-child(4)')

In [45]:
df_record = make_dataframe()

years = soup.select('#cphContents_cphContents_cphContents_ddlSeason_ddlSeason > option')
run_dict = {}
for idx, year in enumerate(years):
    season = year.text
    print(f"{season}시즌")  # 스크래핑 완료 확인용
    go_to_page(f'#cphContents_cphContents_cphContents_ddlSeason_ddlSeason > option:nth-child({idx+1})')
    
    run_dict[season] = team_record_dict()

2001시즌
1 두산
2 롯데
3 삼성
4 한화
5 현대
6 KIA
7 LG
8 SK
2002시즌
1 두산
2 롯데
3 삼성
4 한화
5 현대
6 KIA
7 LG
8 SK
2003시즌
1 두산
2 롯데
3 삼성
4 한화
5 현대
6 KIA
7 LG
8 SK
2004시즌
1 두산
2 롯데
3 삼성
4 한화
5 현대
6 KIA
7 LG
8 SK
2005시즌
1 두산
2 롯데
3 삼성
4 한화
5 현대
6 KIA
7 LG
8 SK
2006시즌
1 두산
2 롯데
3 삼성
4 한화
5 현대
6 KIA
7 LG
8 SK
2007시즌
1 두산
2 롯데
3 삼성
4 한화
5 현대
6 KIA
7 LG
8 SK
2008시즌
1 두산
2 롯데
3 삼성
4 우리
5 한화
6 KIA
7 LG
8 SK
2009시즌
1 히어로즈
2 두산
3 롯데
4 삼성
5 한화
6 KIA
7 LG
8 SK
2010시즌
1 넥센
2 두산
3 롯데
4 삼성
5 한화
6 KIA
7 LG
8 SK
2011시즌
1 넥센
2 두산
3 롯데
4 삼성
5 한화
6 KIA
7 LG
8 SK
2012시즌
1 넥센
2 두산
3 롯데
4 삼성
5 한화
6 KIA
7 LG
8 SK
2013시즌
1 넥센
2 두산
3 롯데
4 삼성
5 한화
6 KIA
7 LG
8 NC
9 SK
2014시즌
1 넥센
2 두산
3 롯데
4 삼성
5 한화
6 KIA
7 LG
8 NC
9 SK
2015시즌
1 넥센
2 두산
3 롯데
4 삼성
5 한화
6 KIA
7 KT
8 LG
9 NC
10 SK
2016시즌
1 넥센
2 두산
3 롯데
4 삼성
5 한화
6 KIA
7 KT
8 LG
9 NC
10 SK
2017시즌
1 넥센
2 두산
3 롯데
4 삼성
5 한화
6 KIA
7 KT
8 LG
9 NC
10 SK
2018시즌
1 넥센
2 두산
3 롯데
4 삼성
5 한화
6 KIA
7 KT
8 LG
9 NC
10 SK
2019시즌
1 두산
2 롯데
3 삼성
4 키움
5 한화
6 KIA
7 KT
8 LG
9 NC
10 SK
2020시즌
1 두산
2 롯데
3 삼성

In [46]:
save_to_excel(run_dict, 'run_record2')

In [14]:
################################################################################

In [ ]:
# 처음 작성한 파일과 함수사용한 파일 비교해보기

In [32]:
df_hitter1 = pd.read_excel('hitter_record/2022.xlsx', sheet_name='두산')

In [33]:
df_hitter2 = pd.read_excel('hitter_record2/2022.xlsx', sheet_name='두산')

In [9]:
df_hitter1.compare(df_hitter2)

Empty DataFrame
Columns: []
Index: []

In [47]:
for i in range(1982, 2023):
    df_hitter1 = pd.ExcelFile(f'hitter_record/{i}.xlsx')
    df_hitter2 = pd.ExcelFile(f'hitter_record2/{i}.xlsx')
    
    hitter1_sheet_names = df_hitter1.sheet_names
    hitter2_sheet_names = df_hitter2.sheet_names
    print(f'{i}시즌')
    for item1, item2 in zip(hitter1_sheet_names, hitter2_sheet_names):
        df1 = df_hitter1.parse(item1)
        df2 = df_hitter2.parse(item2)
        
        print(f'{item1}, {item2}')
        print(df1.compare(df2))
        print('='*10)

1982시즌
롯데, 롯데  
Empty DataFrame
Columns: []
Index: []
삼미, 삼미  
Empty DataFrame
Columns: []
Index: []
삼성, 삼성  
Empty DataFrame
Columns: []
Index: []
해태, 해태  
Empty DataFrame
Columns: []
Index: []
MBC, MBC   
Empty DataFrame
Columns: []
Index: []
OB, OB    
Empty DataFrame
Columns: []
Index: []
1983시즌
롯데, 롯데  
Empty DataFrame
Columns: []
Index: []
삼미, 삼미  
Empty DataFrame
Columns: []
Index: []
삼성, 삼성  
Empty DataFrame
Columns: []
Index: []
해태, 해태  
Empty DataFrame
Columns: []
Index: []
MBC, MBC   
Empty DataFrame
Columns: []
Index: []
OB, OB    
Empty DataFrame
Columns: []
Index: []
1984시즌
롯데, 롯데  
Empty DataFrame
Columns: []
Index: []
삼미, 삼미  
Empty DataFrame
Columns: []
Index: []
삼성, 삼성  
Empty DataFrame
Columns: []
Index: []
해태, 해태  
Empty DataFrame
Columns: []
Index: []
MBC, MBC   
Empty DataFrame
Columns: []
Index: []
OB, OB    
Empty DataFrame
Columns: []
Index: []
1985시즌
롯데, 롯데
Empty DataFrame
Columns: []
Index: []
삼성, 삼성  
Empty DataFrame
Columns: []
Index: []
청보, 청보
Empty DataFr

ValueError: Can only compare identically-labeled DataFrame objects

In [46]:
hitter_96 = pd.ExcelFile('hitter_record/1996.xlsx')
hitter_96.sheet_names

['롯데', '삼성', '쌍방울', '한화', '해태', '현대', 'LG', 'OB']

In [48]:
pd.read_excel('hitter_record/1996.xlsx', sheet_name='삼성')

,순위,선수명,팀명,AVG,G,PA,AB,H,2B,3B,HR,RBI,SB,CS,BB,HBP,SO,GDP,E
0,1,양준혁,삼성,0.346,126,530,436,151,33,2,28,87,23,18,82,6,76,4,2
1,2,이승엽,삼성,0.303,122,504,459,139,32,6,9,76,4,1,34,5,42,10,5
2,3,이만수,삼성,0.285,97,222,193,55,12,1,6,38,2,2,14,10,25,5,0
3,4,김승관,삼성,0.279,23,75,68,19,11,0,0,10,0,1,6,0,9,3,0
4,5,이동수,삼성,0.266,94,277,241,64,13,2,8,38,1,2,27,6,75,6,5
5,6,이종두,삼성,0.262,51,156,130,34,3,0,2,16,9,4,20,2,22,1,1
6,7,최익성,삼성,0.252,57,155,131,33,7,0,4,16,3,5,12,7,21,2,1
7,8,박선일,삼성,0.250,43,79,76,19,5,0,1,7,0,1,3,0,16,1,1
8,9,정경배,삼성,0.247,75,220,194,48,7,2,1,19,4,3,14,1,46,7,12
9,10,이중화,삼성,0.247,65,174,154,38,2,1,2,11,6,2,12,1,18,4,2


In [50]:
pd.read_excel('hitter_record2/1996.xlsx', sheet_name='롯데')

,순위,선수명,팀명,AVG,G,PA,AB,H,2B,3B,HR,RBI,SB,CS,BB,HBP,SO,GDP,E
0,1,이종운,롯데,0.474,6,21,19,9,1,0,0,3,1,0,0,0,2,0,0
1,2,유필선,롯데,0.429,17,17,14,6,1,0,0,2,0,0,1,2,5,0,0
2,3,정호진,롯데,0.375,3,8,8,3,1,0,0,1,0,0,0,0,0,1,1
3,4,김영일,롯데,0.333,41,72,66,22,3,0,0,3,1,1,3,1,3,3,1
4,5,김응국,롯데,0.321,126,518,446,143,22,6,9,64,26,7,65,2,61,11,2
5,6,임수혁,롯데,0.311,113,431,373,116,18,2,11,76,4,5,46,3,46,11,2
6,7,박정태,롯데,0.309,94,405,366,113,25,1,5,61,2,0,32,3,39,11,9
7,8,배정훈,롯데,0.294,12,17,17,5,0,0,0,0,0,1,0,0,6,2,0
8,9,김종훈,롯데,0.290,89,269,238,69,8,5,4,25,15,4,20,5,50,3,0
9,10,박현승,롯데,0.288,108,410,354,102,22,3,8,51,10,6,40,5,55,13,13


In [ ]:
# 일단 다르니까 원본에서 작업...